In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import keras
import numpy as np # linear algebra
import pandas as pd
import tensorflow as tf
from cv2 import cv2
import random
#from keras import bakend as k
from PIL import Image
from keras.utils.np_utils import to_categorical
from keras.losses import binary_crossentropy
from keras.objectives import mean_squared_error,mean_absolute_percentage_error
import keras
from keras.callbacks import *
import matplotlib.pyplot as plt
from keras.applications import MobileNetV2
from keras.optimizers import Adam, RMSprop, SGD
from keras.models import load_model
from sklearn.model_selection import KFold,train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Dropout, Conv2D,Conv2DTranspose, BatchNormalization, Activation,AveragePooling2D,MaxPooling2D,Flatten,GlobalAveragePooling2D, Input, Concatenate, MaxPool2D, Add, UpSampling2D, LeakyReLU,ZeroPadding2D
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau,LearningRateScheduler

In [ ]:
df_train=pd.read_csv("/kaggle/input/landmark-recognition-2020/train.csv")

In [ ]:
df_train.head(5)

In [ ]:
df_train["img"]=df_train.id.str[0]+"/"+df_train.id.str[1]+"/"+df_train.id.str[2]+"/"+df_train.id+".jpg"

In [ ]:
df_train.head(5)

In [ ]:
df_train["label"]=df_train["landmark_id"].values.astype("str")

In [ ]:
df_train.head(5)

In [ ]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)
train_generator=datagen.flow_from_dataframe(
dataframe=df_train,
directory="/kaggle/input/landmark-recognition-2020/train/",
x_col="img",
y_col="label",
subset="training",
batch_size=32,
shuffle=True,
class_mode="categorical",
color_mode="rgb",
target_size=(32,32)
)
valid_generator=datagen.flow_from_dataframe(
dataframe=df_train,
directory="/kaggle/input/landmark-recognition-2020/train/",
x_col="img",
y_col="label",
subset="validation",
batch_size=32,
shuffle=True,
color_mode="rgb",
class_mode="categorical",
target_size=(128,128)
)



In [ ]:
import torch

In [ ]:
from random import choices, sample
from collections import Counter

from plotly import graph_objects as go

from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import Sampler
import albumentations as A



In [ ]:
import torchvision.models as models

In [ ]:

model_l=models.resnext50_32x4d(pretrained=True)
for param in model_l.parameters():
    param.requires_grad = False
    
head = nn.Sequential(
    nn.Linear(1000, 512),
    nn.ReLU(),
    nn.Linear(512, 100),
)
model=nn.Sequential(
    model_l,
    head,
)

In [ ]:
category_n=1
category=0
output_layer_n=category_n+4
def conv(x,y,k,s):
      x=Conv2D(y,kernel_size=k,strides=s,padding='same')(x)
      x=BatchNormalization()(x)
      x=LeakyReLU(alpha=0.1)(x)
      return x
def conv1(a,b,c,d):
      b=Conv2DTranspose(c,kernel_size=2,strides=2,padding='valid',use_bias=False)(b)
      b=BatchNormalization()(b)
      b=LeakyReLU(alpha=0.1)(b)
      x=Concatenate()([a,b])
      x=Conv2D(d,kernel_size=1,strides=1,padding='same')(x)
      x=BatchNormalization()(x)
      x=LeakyReLU(alpha=0.1)(x)
      return x
def adding(a,b):
     x=conv(a,b,3,1)
     x=conv(x,b,3,1)
     x=Add()([x,a])
     return x

In [ ]:
def create_model(input_shape):
     input_layer=Input(input_shape)
     input_layer_1=AveragePooling2D(2)(input_layer)
     input_layer_2=AveragePooling2D(2)(input_layer_1)
     x_0=conv(input_layer,16,3,2)
     concat1=Concatenate()([x_0,input_layer_1])
     x_1=conv(concat1,32,3,2)
     concat2=Concatenate()([x_1,input_layer_2])
     x_2=conv(concat2,64,3,2)
     x=conv(x_2,64,3,1)
     x=adding(x,64)
     x=adding(x,64)
     x_3=conv(x,128,3,2)
     x=conv(x_3,128,3,1)
     x=adding(x,128)
     x=adding(x,128)
     x=adding(x,128)
     x_4=conv(x,256,3,2)
     x=conv(x_4,256,3,1)
     x=adding(x,256)
     x=adding(x,256)
     x=adding(x,256)
     x=adding(x,256)
     x=adding(x,256)
     x_5=conv(x,512,3,2)
     x=conv(x_5,512,3,1)
     x=adding(x,512)
     x=adding(x,512)
     x=adding(x,512)
     x_1= conv(x_1, output_layer_n, 1, 1)
     x_1 = conv1(x_1, x_2, output_layer_n, output_layer_n)
     x_2= conv(x_2, output_layer_n, 1, 1)
     x_2 = conv1(x_2, x_3, output_layer_n, output_layer_n)
     x_1 = conv1(x_1, x_2, output_layer_n, output_layer_n)
     x_3= conv(x_3, output_layer_n, 1, 1)
     x_3 = conv1(x_3, x_4, output_layer_n, output_layer_n) 
     x_2 = conv1(x_2, x_3, output_layer_n, output_layer_n)
     x_1 = conv1(x_1, x_2, output_layer_n, output_layer_n)
     x_4= conv(x_4, output_layer_n, 1, 1)
     x=conv(x, output_layer_n, 1, 1)
     x= UpSampling2D(size=(2, 2))(x)
     x = Concatenate()([x, x_4])
     x=conv(x, output_layer_n, 3, 1)
     x= UpSampling2D(size=(2, 2))(x)
     x = Concatenate()([x, x_3])
     x=conv(x, output_layer_n, 3, 1)
     x= UpSampling2D(size=(2, 2))(x)
     x = Concatenate()([x, x_2])
     x=conv(x, output_layer_n, 3, 1)
     x= UpSampling2D(size=(2, 2))(x) 
     x = Concatenate()([x, x_1])
     x=Conv2D(output_layer_n, kernel_size=3, strides=1, padding="same")(x)
     out = Dense(81313,activation="softmax")(x)
     model=Model(input_layer,out)
     return model

In [ ]:
model=MobileNetV2(
    input_shape=(224,224,3),
    alpha=1.0,
    include_top=False,
    input_tensor=None,
    pooling=None,
    classes=81313,
    classifier_activation="softmax",
)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1026))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(81313, activation='softmax'))

In [ ]:
#model_x=load_model("/kaggle/input/common-keras-pretrained-models/MobileNetV2.h5")
#model=create_model(input_shape=(128,128,3))
learning_rate=0.001
lrr=ReduceLROnPlateau(monitor="accuracy",patience=2,verbose=1,factor=0.5,min_lr=0.0000001)
model.compile(optimizer=Adam(lr=learning_rate),loss="categorical_crossentropy",metrics=["accuracy"])
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size
hist=model.fit(train_generator,
                    steps_per_epoch=step_size_train,
                    validation_data=valid_generator,
                    validation_steps=step_size_valid,
                    epochs=1,
              verbose=1,callbacks=[lrr]
)
model.save_weights("save_weights.h5")

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import os
import PIL

In [ ]:
df_train=pd.read_csv("/kaggle/input/landmark-recognition-2020/train.csv")
train_df=pd.read_csv("/kaggle/input/landmark-recognition-2020/train.csv")
df_train["img"]=df_train.id.str[0]+"/"+df_train.id.str[1]+"/"+df_train.id.str[2]+"/"+df_train.id+".jpg"
df_train.drop(["id"],inplace=True,axis=1)
df_train.head(5)

In [ ]:
print((train_df.keys))
number=1580470

In [ ]:
def main():
 NUM_PUBLIC_TRAIN_IMAGES=1580470
 if number == NUM_PUBLIC_TRAIN_IMAGES:
    save_submission()
    return
 _,pred=predictions(df_train)
 save_submission(pred)


In [ ]:
def save_submission(prediction=None):
     if prediction is None:
        shutil.copyfile(os.path.join(DATASET_DIR, 'sample_submission.csv'), 'submission.csv')
        return
     with open('submission.csv', 'w') as submission_csv:
      csv_writer = csv.DictWriter(submission_csv, fieldnames=['id', 'landmarks'])
      csv_writer.writeheader()
      for image_id, prediction in predictions.items():
       label = prediction['class']
       score = prediction['score']
       csv_writer.writerow({'id': image_id, 'landmarks': f'{label} {score}'})


In [ ]:
def predictions(df_t):
    test_ids,test_embeddings=extract()